In [39]:
# Load tools needed for project 1

import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import panel as pn
import os
from pathlib import Path
from dotenv import load_dotenv
%matplotlib inline  

ModuleNotFoundError: No module named 'plotly'

In [40]:
pn.extension('plotly')

NameError: name 'pn' is not defined

In [41]:
import hvplot.pandas

ModuleNotFoundError: No module named 'hvplot.pandas'

In [42]:
load_dotenv()
mapbox = os.getenv("mapbox_api_key")
px.set_mapbox_access_token(mapbox)

NameError: name 'load_dotenv' is not defined

In [43]:
# Set Variables

In [44]:
# Load Data


In [45]:
# Import Data

nfl_data = pd.DataFrame([])
years_start = 1996
years_end = 2022
for year in range(years_start,years_end):
        url = f'https://www.pro-football-reference.com/years/{year}/preseason_odds.htm'
        df = pd.read_html(url)[0]
        df['year'] = year
        nfl_data = nfl_data.append(df, ignore_index=True, sort=False)
        
nfl_data.columns = ['Team', 'Super Bowl Odds', 'W/l Over-Under', 'Record', 'Year']

nfl_data.head()



,Team,Super Bowl Odds,W/l Over-Under,Record,Year
0,Arizona Cardinals,12000,5.5,7-9 (over),1996
1,Atlanta Falcons,4000,8.0,3-13 (under),1996
2,Baltimore Ravens,3000,7.5,4-12 (under),1996
3,Buffalo Bills,1500,8.5,10-6 (over),1996
4,Carolina Panthers,8000,5.5,12-4 (over),1996


In [46]:
superbowl = pd.read_csv(
    "superbowl_data.csv")
superbowl['Season Year'] = superbowl['Year'] - 1
#superbowl.iloc[:,0] = superbowl.iloc[:,0].shift(1)
superbowl.head()

,Date,Year,SB,Winner,Winner Pts,Loser,Loser Pts,MVP,Stadium,City,State,Season Year
0,Feb 7,2021,LV (55),Tampa Bay Buccaneers,31,Kansas City Chiefs,9,Tom Brady,Raymond James Stadium,Tampa,Florida,2020
1,Feb 2,2020,LIV (54),Kansas City Chiefs,31,San Francisco 49ers,20,Patrick Mahomes,Hard Rock Stadium,Miami Gardens,Florida,2019
2,Feb 3,2019,LIII (53),New England Patriots,13,Los Angeles Rams,3,Julian Edelman,Mercedes-Benz Stadium,Atlanta,Georgia,2018
3,Feb 4,2018,LII (52),Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota,2017
4,Feb 5,2017,LI (51),New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas,2016


In [47]:
nfl_data.sort_values(['Year','Super Bowl Odds'], ascending=True, inplace=True)
nfl_data.set_index('Year')


,Team,Super Bowl Odds,W/l Over-Under,Record
Year,,,,
1996,San Francisco 49ers,250,12.0,12-4 (push)
1996,Dallas Cowboys,600,10.5,10-6 (under)
1996,Green Bay Packers,800,10.0,13-3 (over)
1996,Pittsburgh Steelers,1000,10.0,10-6 (push)
1996,Kansas City Chiefs,1200,9.0,9-7 (push)
...,...,...,...,...
2021,Jacksonville Jaguars,12500,6.5,3-14
2021,Cincinnati Bengals,15000,6.5,10-7
2021,New York Jets,15000,6.0,4-13


In [64]:
# Strategy 1 - $100 on favorite
def top1(x):
    dftop1 = nfl_data[nfl_data['Year'] == x]
    return dftop1.head(1)
# Strategy 2 - $100 on Top 3
def top2(x):
    dftop2 = nfl_data[nfl_data['Year'] == x]
    return dftop2.head(2)

# Strategy 3 - $100 on Top 5def top5(x):
def top5(x):
    dftop5 = nfl_data[nfl_data['Year'] == x]
    return dftop5.head(5)

In [49]:
top5(2019)

,Team,Super Bowl Odds,W/l Over-Under,Record,Year
747,New England Patriots,400,11.5,12-4 (over),2019
742,Kansas City Chiefs,600,10.5,12-4 (over),2019
748,New Orleans Saints,1000,10.5,13-3 (over),2019
732,Chicago Bears,1200,9.0,8-8 (under),2019
752,Philadelphia Eagles,1200,10.0,9-7 (under),2019


In [50]:
def xy(yr,strategy):
    superbowl_winners = superbowl[superbowl['Year'] == yr]
    if strategy == 1:
        if any(team==superbowl_winners['Winner'].item() for team in top1(yr)['Team']):
            return superbowl_winners['Winner'].item()
        else:
            return False
    elif strategy == 2:
        if any(team==superbowl_winners['Winner'].item() for team in top2(yr)['Team']):
            return superbowl_winners['Winner'].item()
        else:
            return False
    elif strategy == 3:
        if any(team==superbowl_winners['Winner'].item() for team in top5(yr)['Team']):
            return superbowl_winners['Winner'].item()
        else:
            return False

In [51]:
def odds(yr, strategy):
    if strategy == 1:
        superbowl_odds = top1(yr)[top1(yr)['Team'] == xy(yr,strategy)]
        return superbowl_odds['Super Bowl Odds'].item()
    elif strategy == 2:
        superbowl_odds = top2(yr)[top2(yr)['Team'] == xy(yr,strategy)]
        return superbowl_odds['Super Bowl Odds'].item()
    elif strategy == 3:
        superbowl_odds = top5(yr)[top5(yr)['Team'] == xy(yr,strategy)]
        return superbowl_odds['Super Bowl Odds'].item()


In [52]:
def main(yr,strategy):
    while xy(yr,strategy) != False:
        if strategy == 1:
            return ((odds(yr,strategy))*1) +100
        elif strategy == 2:
            return ((odds(yr,strategy))*.5) -50
        elif strategy == 3:
            return ((odds(yr,strategy))*.2) -80
    else:
        return -100 

In [79]:
strategies = [3,2,1]
df_combined = superbowl
for strategy in strategies:
    output = []
    for yr in range(years_start,years_end):
        output.append(main(yr,strategy))
    output.reverse()
    df = pd.DataFrame(output)
    df.columns = [f"strat{strategy}"]
    df_combined = pd.concat([df, df_combined], axis=1)
df_combined

,strat1,strat2,strat3,Date,Year,SB,Winner,Winner Pts,Loser,Loser Pts,MVP,Stadium,City,State,Season Year
0,-100.0,300.0,60.0,Feb 7,2021,LV (55),Tampa Bay Buccaneers,31,Kansas City Chiefs,9,Tom Brady,Raymond James Stadium,Tampa,Florida,2020
1,550.0,175.0,10.0,Feb 2,2020,LIV (54),Kansas City Chiefs,31,San Francisco 49ers,20,Patrick Mahomes,Hard Rock Stadium,Miami Gardens,Florida,2019
2,500.0,150.0,0.0,Feb 3,2019,LIII (53),New England Patriots,13,Los Angeles Rams,3,Julian Edelman,Mercedes-Benz Stadium,Atlanta,Georgia,2018
3,-100.0,-100.0,-100.0,Feb 4,2018,LII (52),Philadelphia Eagles,41,New England Patriots,33,Nick Foles,U.S. Bank Stadium,Minneapolis,Minnesota,2017
4,375.0,87.5,-25.0,Feb 5,2017,LI (51),New England Patriots,34,Atlanta Falcons,28,Tom Brady,NRG Stadium,Houston,Texas,2016
5,-100.0,-100.0,-100.0,Feb 7,2016,50,Denver Broncos,24,Carolina Panthers,10,Von Miller,Levi's Stadium,Santa Clara,California,2015
6,-100.0,-100.0,80.0,Feb 1,2015,XLIX (49),New England Patriots,28,Seattle Seahawks,24,Tom Brady,University of Phoenix Stadium,Glendale,Arizona,2014
7,550.0,175.0,10.0,Feb 2,2014,XLVIII (48),Seattle Seahawks,43,Denver Broncos,8,Malcolm Smith,MetLife Stadium,East Rutherford,New Jersey,2013
8,-100.0,-100.0,-100.0,Feb 3,2013,XLVII (47),Baltimore Ravens,34,San Francisco 49ers,31,Joe Flacco,Mercedes-Benz Superdome,New Orleans,Louisiana,2012
9,-100.0,-100.0,-100.0,Feb 5,2012,XLVI (46),New York Giants,21,New England Patriots,17,Eli Manning,Lucas Oil Stadium,Indianapolis,Indiana,2011


In [84]:
df_strats = df_combined[['Year','strat1','strat2','strat3']]
df_strats 

,Year,strat1,strat2,strat3
0,2021,-100.0,300.0,60.0
1,2020,550.0,175.0,10.0
2,2019,500.0,150.0,0.0
3,2018,-100.0,-100.0,-100.0
4,2017,375.0,87.5,-25.0
5,2016,-100.0,-100.0,-100.0
6,2015,-100.0,-100.0,80.0
7,2014,550.0,175.0,10.0
8,2013,-100.0,-100.0,-100.0
9,2012,-100.0,-100.0,-100.0


In [ ]:
# Charting - Individual Bar Graph Strat 1, 2, & 3
main(2018,1)
# Charting - Combined Bar Graph Total Wins Over Years

# Charting - Combine Bar Graph Total $ Won or Lost Over Years

In [ ]:
# Correlation Analysis

# 1. Correlation between end of season preseason leaders vs. Making SB
# Charting - Pie Chart? Scatter Plot?

In [ ]:
# 2. Correlation between end of season records of preseason favorites vs SB Teams
# Charting - Side by side - Bar Graph?

In [ ]:
# 3. Correlation between preseason wins vs regular season success  
# Charting - Side by side - Bar Graph?

In [ ]:
# Post Season Analysis
# Distance Travel by Teams wins and losses
# Charting - Heatmap?

# Post Season odds